In [1]:
import sys
import os# Navigate up one level to the parent directory and append it to sys.path
import importlib
import requests

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src import homers
import plotly.express as px
from src.config import LOGO_PATH,  BETTING_PATH

POWER_RATING_PATH = BETTING_PATH / 'power-ratings.pkl'

In [19]:
SEASON, WEEK = 2023, 11
HFA = 1.5
AVG_POINTS = 21.5

In [24]:
def read_power(season, week):
    power_df = pd.read_clipboard()
    assert len(power_df) == 32
    power_df['season']=season
    power_df['week']=week
    power_df['power_id'] = power_df.apply(lambda row: f'{row.season}_{row.week}_{row.team}', axis=1)
    power_df = power_df.set_index('power_id').sort_values(by=['ovr', 'off'], ascending=False)
    return power_df

In [30]:
current_week = read_power(SEASON, WEEK)
pwr = pd.read_pickle(POWER_RATING_PATH)
pwr = pd.concat([pwr, current_week])
pwr.to_pickle(POWER_RATING_PATH)

In [31]:
sched = nfl.import_schedules([SEASON])

In [32]:
sched = sched[sched['week']==WEEK][['game_id', 'away_team', 'home_team']].copy().set_index('game_id')
sched.head()

,away_team,home_team
game_id,,
2023_11_CIN_BAL,CIN,BAL
2023_11_DAL_CAR,DAL,CAR
2023_11_PIT_CLE,PIT,CLE
2023_11_CHI_DET,CHI,DET
2023_11_LAC_GB,LAC,GB


In [33]:
predict_home_score = lambda row: AVG_POINTS + row.home_off - row.away_def + HFA / 2
predict_away_score = lambda row: AVG_POINTS + row.away_off - row.home_def - HFA / 2

In [36]:
# guess the lines
gtl = pd.merge(sched, pwr[(pwr['week'] == WEEK) & (pwr['season']==SEASON)][['team', 'ovr', 'off', 'def']], left_on='away_team', right_on='team').drop(columns=['team']).rename(columns={'ovr': 'away_ovr', 'off': 'away_off', 'def': 'away_def'})
gtl = pd.merge(gtl, pwr[(pwr['week'] == WEEK) & (pwr['season']==SEASON)][['team', 'ovr', 'off', 'def']], left_on='home_team', right_on='team').drop(columns=['team']).rename(columns={'ovr': 'home_ovr', 'off': 'home_off', 'def': 'home_def'})
gtl['hfa'] = HFA
gtl['pred_away_score'] = gtl.apply(predict_away_score, axis=1)
gtl['pred_home_score'] = gtl.apply(predict_home_score, axis=1)
gtl['spread'] = gtl['pred_home_score'] - gtl['pred_away_score']
gtl['total'] = gtl['pred_home_score'] + gtl['pred_away_score']
gtl = gtl[['away_team', 'home_team', 'spread', 'total', 'hfa', 'pred_away_score', 'pred_home_score', 'away_ovr', 'home_ovr']].copy()
gtl['game_id'] = gtl.apply(lambda row: f'{SEASON}_{WEEK}_{row.away_team}_{row.home_team}', axis=1)
gtl = gtl.set_index('game_id')
gtl

,away_team,home_team,spread,total,hfa,pred_away_score,pred_home_score,away_ovr,home_ovr
game_id,,,,,,,,,
2023_11_CIN_BAL,CIN,BAL,2.1,45.0,1.5,21.45,23.55,5.4,6.0
2023_11_DAL_CAR,DAL,CAR,-9.6,42.3,1.5,25.95,16.35,5.0,-6.1
2023_11_PIT_CLE,PIT,CLE,5.2,35.3,1.5,15.05,20.25,-1.4,2.3
2023_11_CHI_DET,CHI,DET,9.2,45.1,1.5,17.95,27.15,-4.6,3.1
2023_11_LAC_GB,LAC,GB,-3.8,46.1,1.5,24.95,21.15,1.6,-3.7
2023_11_ARI_HOU,ARI,HOU,5.4,48.1,1.5,21.35,26.75,-3.8,0.1
2023_11_TEN_JAX,TEN,JAX,6.3,42.4,1.5,18.05,24.35,-2.7,2.1
2023_11_LV_MIA,LV,MIA,10.7,47.8,1.5,18.55,29.25,-3.8,5.4
2023_11_NYG_WAS,NYG,WAS,7.0,36.3,1.5,14.65,21.65,-7.8,-2.3


# Evaluate Movement between Weeks

In [11]:

previous_power = pd.read_clipboard()
previous_power

,team,ovr,off,def
0,KC,6.5,4.1,2.4
1,PHI,6.2,4.4,1.8
2,SF,5.9,3.1,2.8
3,BAL,5.1,2.5,2.6
4,MIA,4.9,5.6,-0.7
5,DAL,4.8,2.5,2.3
6,CIN,4.7,3.2,1.5
7,BUF,4.1,3.6,0.5
8,JAX,2.7,1.8,0.9
9,DET,2.5,2.6,-0.1


In [12]:
movement = pd.merge(power_df, previous_power, on='team', suffixes=('_cur', '_prev'))
for col in ['ovr', 'off', 'def']:
    movement[f'{col}_dif'] = movement[f'{col}_cur'] - movement[f'{col}_prev']
movement

,team,ovr_cur,off_cur,def_cur,season,week,ovr_prev,off_prev,def_prev,ovr_dif,off_dif,def_dif
0,KC,6.8,4.3,2.5,2023,10,6.5,4.1,2.4,0.3,0.2,0.1
1,PHI,6.4,4.5,1.9,2023,10,6.2,4.4,1.8,0.2,0.1,0.1
2,BAL,6.3,3.1,3.2,2023,10,5.1,2.5,2.6,1.2,0.6,0.6
3,SF,5.9,3.1,2.8,2023,10,5.9,3.1,2.8,0.0,0.0,0.0
4,MIA,5.5,5.4,0.1,2023,10,4.9,5.6,-0.7,0.6,-0.2,0.8
5,CIN,5.4,3.8,1.6,2023,10,4.7,3.2,1.5,0.7,0.6,0.1
6,DAL,4.6,2.4,2.2,2023,10,4.8,2.5,2.3,-0.2,-0.1,-0.1
7,BUF,3.7,3.2,0.5,2023,10,4.1,3.6,0.5,-0.4,-0.4,0.0
8,DET,2.9,2.6,0.3,2023,10,2.5,2.6,-0.1,0.4,0.0,0.4
9,JAX,2.7,1.8,0.9,2023,10,2.7,1.8,0.9,0.0,0.0,0.0


In [13]:
# utils.download_team_pngs()

In [14]:
col = 'ovr_cur'

# Create the bar chart
fig = px.bar(movement, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(
    height=1000,
    width=800,
    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in movement.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Show the chart
fig.show()

In [16]:
col = 'ovr_dif'

# Create the bar chart
fig = px.bar(movement, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(
    height=1000,
    width=800,
    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in movement.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Show the chart
fig.show()

In [15]:


fig = px.scatter(power_df, x='off', y='def', opacity=0, color='team', color_discrete_map=utils.team_unique_colors)
fig.update_layout(
    height=600,
    width=1000,

    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)
# Iterate through the data and add logos to the chart
for index, row in power_df.iterrows():
    team = row['team']
    scale = 0.9
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row['off'],  # Adjust the position
            y=row['def'],
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

fig.show()